In [2]:
import os
import pandas as pd
import csv
import json
import time

from datetime import datetime
from retrying import retry
from retrying import RetryError
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY=[]
for x in range(2,8):
    if (x==2):
        DEVELOPER_KEY.append(os.environ.get('API_CODE'))
        DEVELOPER_KEY.append(os.environ.get('API_CODE2'))
    else:
        DEVELOPER_KEY.append(os.environ.get('API_CODE'+str(x)))
                             
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[0])

In [3]:
def is_503_error(exception):
    is503=False
    if (isinstance(exception, HttpError)):
        if (exception.resp.status==503):
            is503=True
    return (is503)

In [4]:
@retry(stop_max_attempt_number=7,retry_on_exception=is_503_error, 
       wrap_exception=True,wait_random_min=1000, wait_random_max=2000)

def comment_thread_list(pageToken, videoId):

    commentList=service.commentThreads().list(
        part='snippet',
        videoId=videoId,
        maxResults=100,
        order='relevance',
        pageToken=pageToken
    ).execute()
    
    return commentList

In [5]:
def comments_by_video(videoId,videoYear,writer,ferror):
    this_quota=0
    pageToken=""
    c=0
    
    #gets information until there is no next page
    while (pageToken!='error') and (c<=50):
        
        try:
            commentList=comment_thread_list(pageToken,videoId)
            this_quota+=20
            # track quota inside function
#             print(this_quota)
        except RetryError as e:
#             print(e)
            with open(ferror, 'a') as f:
                f.write("max unsuccessful attempts reached"+" videoId:"+videoId+'\n')
                this_quota+=30
                # track quota inside function
#                 print('error:'+str(this_quota))
                break
            continue

        #save list of videos if the list is not empty
        if (commentList.get("items",[])!=[]):
            
            for item in commentList.get("items",[]):
                year=2019
                dictio={}
                dictio['videoId']= videoId
                dictio['year']=videoYear
                
                if 'id' in item['snippet']['topLevelComment']:
                    dictio['commentId']=item['snippet']['topLevelComment']['id']
                    
                if 'authorDisplayName' in item['snippet']['topLevelComment']['snippet']:
                    dictio['authorName']=item['snippet']['topLevelComment']['snippet']['authorDisplayName']
                
                if 'authorChannelId' in item['snippet']['topLevelComment']['snippet']:
                    dictio['authorChannelId']=item['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
                
                if 'textDisplay' in item['snippet']['topLevelComment']['snippet']:
                    dictio['textDisplay']=item['snippet']['topLevelComment']['snippet']['textDisplay']
                
                if 'textOriginal' in item['snippet']['topLevelComment']['snippet']:
                    dictio['textOriginal']=item['snippet']['topLevelComment']['snippet']['textOriginal']
                    
                if 'likeCount' in item['snippet']['topLevelComment']['snippet']:
                    dictio['likeCount']=item['snippet']['topLevelComment']['snippet']['likeCount']
                    
                if 'publishedAt' in item['snippet']['topLevelComment']['snippet']:
                    pubAt=item['snippet']['topLevelComment']['snippet']['publishedAt']
                    date=datetime.strptime(pubAt, '%Y-%m-%dT%H:%M:%S.%fZ')
                    year =date.year
                    dictio['pYear']= date.year
                    dictio['pMonth']=date.month
                    dictio['pDay']= date.day
                
                if (year<=2016):
                    writer.writerow(dictio)
                    c+=1

        time.sleep(0.01)
        
        try:
            pageToken=commentList['nextPageToken']
        except KeyError as e:
            pageToken='error'
        
        if this_quota>=180000:
            break
            
    return this_quota

In [6]:
community_num=[9,22,85,76,71,37,87]
data_islp='/media/aruiz/data/videos_data/islp/'
data_comments='/media/aruiz/data/comments_data/'
fieldnames=['videoId','year','commentId','authorName','authorChannelId','textDisplay','textOriginal','likeCount',
            'pYear', 'pMonth','pDay']
fError=os.path.join(data_comments,'errors.txt')
quota=250000
key=0

for ind in range (0, 1):
# for ind in range (2, len(community_num)):
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
    fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
    out1=open(fileName1, 'a')
    writer1 = csv.DictWriter(out1, fieldnames=fieldnames)
    writer1.writeheader()
    fileName2=os.path.join(data_comments,'community'+str(community_num[ind])+'_allNegative.csv')
    out2=open(fileName2, 'a')
    writer2 = csv.DictWriter(out2, fieldnames=fieldnames)
    writer2.writeheader()
    
    df=pd.read_csv(f)
    df=df.fillna(0.0)
    df=df[df['is_letPlay']==True]
    df=df[df['categoryId']==20]      

    for dyear in range (2008,2017):
#     for dyear in range (2014,2017):
        dft=df[df['year']==dyear]
        
        # get the first 100 videos ids
        if len(dft)>100:
            if dyear>=2010:
                # Values to get rating of most liked/disliked videos
                dft['ratio']= dft['likeCount']-dft['dislikeCount']
                dft=dft.sort_values(by=['ratio'],ascending=False)
                dft=dft.reset_index(drop=True)
            
            #get top 100 of most approved videos
            ids=dft.id[0:100]
        else:
            if dyear>=2010:
                # Values to get rating of most liked/disliked videos
                dft['ratio']= dft['likeCount']-dft['dislikeCount']
                dft=dft.sort_values(by=['ratio'],ascending=False)
                dft=dft.reset_index(drop=True)
                
            ids=dft.id

        #writing all approved videos comments
        for i in ids:
            this_quota=comments_by_video(i,dyear,writer1,fError)
            quota+=this_quota
#             print (quota)

            if (quota>=800000):
                quota=0
                key+=1
                if (key==7):
                    key=0                        
                service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[key])
                continue

        if dyear>=2010:   
            dft=dft[dft['ratio']<0]
            ids=dft.id            
        
            #writing all disapproved videos comments
            for i in ids:
                this_quota=comments_by_video(i,dyear,writer2,fError)
                quota+=this_quota

                if (quota>=800000):
                    quota=0
                    key+=1
                    if (key==7):
                        key=0                        
                    service= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY[key])
                    continue
                    
    out1.close()
    out2.close()

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Clear all files to start again

In [8]:
community_num=[9,22,85,76,71,37,87]
data_islp='/media/aruiz/data/videos_data/islp/'
data_comments='/media/aruiz/data/comments_data/'
fieldnames=['videoId','year','commentId','authorName','authorChannelId','textDisplay','textOriginal','likeCount',
            'publishedAt']
fError=os.path.join(data_comments,'errors.txt')
quota=0

for ind in range (0, len(community_num)):
    f=os.path.join(data_islp,'data_community_'+str(community_num[ind])+'_islp.csv')
#     fileName1=os.path.join(data_comments,'community'+str(community_num[ind])+'_allPositive.csv')
#     out1=open(fileName1, 'a')
#     writer1 = csv.DictWriter(out1, fieldnames=fieldnames)
#     writer1.writeheader()
#     fileName2=os.path.join(data_comments,'community'+str(community_num[ind])+'_allNegative.csv')
#     out2=open(fileName2, 'a')
#     writer2 = csv.DictWriter(out2, fieldnames=fieldnames)
#     writer2.writeheader()
    
#     this_quota=comments_by_video('098F0WJTRK4',2014,writer2,fError)
#     break
    df=pd.read_csv(f)
    df=df.fillna(0.0)
    df=df[df['is_letPlay']==True]
    df=df[df['categoryId']==20]      
    print ("community"+str(community_num[ind])+'\n')

    for dyear in range (2008,2017):
        dft=df[df['year']==dyear]
        print (str(dyear)+ " "+ str(len(dft)))
        
        if dyear>=2010:
            # Values to get rating of most liked/disliked videos
            dft['ratio']= dft['likeCount']-dft['dislikeCount']
            dft=dft.sort_values(by=['ratio'],ascending=False)
            dft=dft.reset_index(drop=True)
            dft=dft[dft['ratio']<0]
            print("negative: "+str(len(dft)))

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


community9

2008 354
2009 1468
2010 4006
negative: 5
2011 7362
negative: 17
2012 16389
negative: 51
2013 19349


/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


negative: 40
2014 26544
negative: 50
2015 21995
negative: 147
2016 26343
negative: 72
community22

2008 11
2009 54
2010 685
negative: 10
2011 2986
negative: 19
2012 6104
negative: 16
2013 8580
negative: 44
2014 11219
negative: 34
2015 10600
negative: 139
2016 10949
negative: 36
community85

2008 3
2009 34
2010 70
negative: 2
2011 976
negative: 10
2012 2059
negative: 16
2013 2446
negative: 21
2014 2488
negative: 9
2015 2220
negative: 8
2016 1955
negative: 3
community76

2008 0
2009 0
2010 0
negative: 0
2011 21
negative: 0
2012 87
negative: 0
2013 90
negative: 1
2014 138
negative: 1
2015 454
negative: 2
2016 580
negative: 3
community71

2008 1
2009 1
2010 34
negative: 0
2011 170
negative: 0
2012 310
negative: 2
2013 880
negative: 6
2014 1065
negative: 15
2015 962
negative: 10
2016 899
negative: 25
community37

2008 0
2009 0
2010 0
negative: 0
2011 35
negative: 0
2012 199
negative: 1
2013 254
negative: 1
2014 160
negative: 0
2015 272
negative: 0
2016 527
negative: 1
community87

2008 0
20

In [29]:
def method():
    i=0
    pageToken=""
    c=0

    while pageToken!='error' and c<=10:
            i+=1
            if i>=51:
                break
            if i==50:
                c+=1
                pageToken='error'
            
    return c,pageToken

In [30]:
c,pg=method()

In [32]:
pg

'error'